# functions: write_result

[write_result](../api/functions.rst#nornir_utils.plugins.functions.write_result) is a function that writes result to file. This function is based on [print_result](../api/functions.rst#nornir_utils.plugins.functions.print_result) function with `file` argument:

In [1]:
from nornir import InitNornir

def dummy_task(task):
    return "hi!!!"

nr = InitNornir(
    inventory={
        "plugin":"YAMLInventory",
        "options": {
            "host_file": "data/hosts.yaml",
            "group_file": "data/groups.yaml",
            "defaults_file": "data/defaults.yaml",
        }
    }
)

result = nr.run(task=dummy_task)

Now we could write the result easily with the `write_result` function:

In [2]:
from nornir_utils.plugins.functions import write_result

write_result(result, filepath="out_files/write_result/result.txt")

In [3]:
with open("out_files/write_result/result.txt") as f:
    print(f.read())

dummy_task**********************************************************************
* dev1.group_1 ** changed : False **********************************************
vvvv dummy_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
hi!!!
^^^^ END dummy_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev2.group_1 ** changed : False **********************************************
vvvv dummy_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
hi!!!
^^^^ END dummy_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev3.group_2 ** changed : False **********************************************
vvvv dummy_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
hi!!!
^^^^ END dummy_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev4.group_2 ** changed : False **********************************************
vvvv dummy_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv

`filepath` here is path to file, you want to write the result. Any missing directories from the `filepath` argument are created as needed.

## Writing specific data

If the task returns different information, you can also select which ones to write. For instance:

In [4]:
from nornir.core.task import Result

def task_with_data(task):
    return Result(host=task.host, result="Spaghetti monster", sw_char="Jar Jar Binks", food="hawaiian pizza", OS="windows")


fav_result = nr.run(task=task_with_data)

# write only sw_char and food vars
write_result(fav_result, filepath="out_files/write_result/result.txt", vars=["sw_char", "food"])

In [5]:
with open("out_files/write_result/result.txt") as f:
    print(f.read())

task_with_data******************************************************************
* dev1.group_1 ** changed : False **********************************************
vvvv task_with_data ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
Jar Jar Binks
hawaiian pizza
^^^^ END task_with_data ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev2.group_1 ** changed : False **********************************************
vvvv task_with_data ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
Jar Jar Binks
hawaiian pizza
^^^^ END task_with_data ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev3.group_2 ** changed : False **********************************************
vvvv task_with_data ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
Jar Jar Binks
hawaiian pizza
^^^^ END task_with_data ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev4.group_2 ** changed : False **********************************************
vvvv ta

## Severity

You can also ask `write_result` to write the results only if the severity is equal or superior to the one specified. This is particularly useful if your script is very large and you don't care about intermediate results. For instance:

In [6]:
import logging

def complex_task(task):
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    return "I did a lot of things!!!"

complex_result = nr.run(task=complex_task)

`write_result` will only write `INFO` severity and above by default, so, if we write the result, we should only see the result of the parent task:

In [7]:
write_result(complex_result, filepath="out_files/write_result/result.txt")

In [8]:
with open("out_files/write_result/result.txt") as f:
    print(f.read())

complex_task********************************************************************
* dev1.group_1 ** changed : False **********************************************
vvvv complex_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of things!!!
^^^^ END complex_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev2.group_1 ** changed : False **********************************************
vvvv complex_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of things!!!
^^^^ END complex_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev3.group_2 ** changed : False **********************************************
vvvv complex_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of things!!!
^^^^ END complex_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev4.group_2 ** changed : False **********************************************
vvvv complex_task *

If you want to write all the tasks for debugging purposes you could specify it with:

In [9]:
write_result(complex_result, filepath="out_files/write_result/result.txt", severity_level=logging.DEBUG)

In [10]:
with open("out_files/write_result/result.txt") as f:
    print(f.read())

complex_task********************************************************************
* dev1.group_1 ** changed : False **********************************************
vvvv complex_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of things!!!
---- dummy_task ** changed : False --------------------------------------------- DEBUG
hi!!!
---- dummy_task ** changed : False --------------------------------------------- DEBUG
hi!!!
---- dummy_task ** changed : False --------------------------------------------- DEBUG
hi!!!
---- dummy_task ** changed : False --------------------------------------------- DEBUG
hi!!!
---- dummy_task ** changed : False --------------------------------------------- DEBUG
hi!!!
^^^^ END complex_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev2.group_1 ** changed : False **********************************************
vvvv complex_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of

## No_errors

If some `Result` objects have exceptions, then you can exclude them from the file:

In [11]:
def task_with_exception(task):
    return Result(host=task.host, result="Something went wrong", exception=Exception())


def task_without_exception(task):
    return "All is fine"

def task(task):
    task.run(task=task_with_exception)
    task.run(task=task_without_exception)
    return "What's happening?"

    
result_with_exception = nr.run(task=task)

write_result(result_with_exception, filepath="out_files/write_result/result_with_exception.txt")

In [12]:
with open("out_files/write_result/result_with_exception.txt") as f:
    print(f.read())

task****************************************************************************
* dev1.group_1 ** changed : False **********************************************
vvvv task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
What's happening?
---- task_with_exception ** changed : False ------------------------------------ INFO
Something went wrong
---- task_without_exception ** changed : False --------------------------------- INFO
All is fine
^^^^ END task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev2.group_1 ** changed : False **********************************************
vvvv task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
What's happening?
---- task_with_exception ** changed : False ------------------------------------ INFO
Something went wrong
---- task_without_exception ** changed : False --------------------------------- INFO
All is fine
^^^^ END task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [13]:
write_result(result_with_exception, filepath="out_files/write_result/result_without_exception.txt", no_errors=True)

In [14]:
with open("out_files/write_result/result_without_exception.txt") as f:
    print(f.read())

task****************************************************************************
* dev1.group_1 ** changed : False **********************************************
vvvv task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
What's happening?
---- task_without_exception ** changed : False --------------------------------- INFO
All is fine
^^^^ END task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev2.group_1 ** changed : False **********************************************
vvvv task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
What's happening?
---- task_without_exception ** changed : False --------------------------------- INFO
All is fine
^^^^ END task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev3.group_2 ** changed : False **********************************************
vvvv task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
What's happening?
---- t

## Write_host

`write_host` argument can help you, if you don't need to write hostnames to file for `MultiResult` or `Result` object. As instance, this can be useful, when there is already hostname in the filename and you don't want to duplicate it.

For `MultiResult`:

In [15]:
write_result(complex_result['dev1.group_1'], filepath="out_files/write_result/dev1.group_1_multiresult.txt", write_host=False)

In [16]:
with open("out_files/write_result/dev1.group_1_multiresult.txt") as f:
    print(f.read())

vvvv complex_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of things!!!
^^^^ END complex_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^



For `Result`:

In [17]:
write_result(complex_result['dev1.group_1'][0], filepath="out_files/write_result/dev1.group_1_result.txt", write_host=False)

In [18]:
with open("out_files/write_result/dev1.group_1_result.txt") as f:
    print(f.read())

---- complex_task ** changed : False ------------------------------------------- INFO
I did a lot of things!!!



## Append

`append` argument allows you to use `a+` or `w+` (default mode) writing mode. As instance:

In [19]:
write_result(fav_result, filepath="out_files/write_result/result.txt", append=True)

In [20]:
with open("out_files/write_result/result.txt") as f:
    print(f.read())

complex_task********************************************************************
* dev1.group_1 ** changed : False **********************************************
vvvv complex_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of things!!!
---- dummy_task ** changed : False --------------------------------------------- DEBUG
hi!!!
---- dummy_task ** changed : False --------------------------------------------- DEBUG
hi!!!
---- dummy_task ** changed : False --------------------------------------------- DEBUG
hi!!!
---- dummy_task ** changed : False --------------------------------------------- DEBUG
hi!!!
---- dummy_task ** changed : False --------------------------------------------- DEBUG
hi!!!
^^^^ END complex_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev2.group_1 ** changed : False **********************************************
vvvv complex_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of